In [1]:
# Importing the necessary packages

import pandas as pd
import numpy as np

import tensorflow as tf
import h5py

from keras.utils import to_categorical

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD
from keras.backend import clear_session
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
# Mounting the google drive

from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# 1. Understand the basic Image Classification pipeline and the data-driven approach (train/predict stages)

In [3]:
# Reading the dataset

images_of_numbers_file = h5py.File("/gdrive/My Drive/Colab Notebooks/SVHN_single_grey1.h5", "r+")
list_of_keys = list(images_of_numbers_file.keys())
list_of_keys

['X_test', 'X_train', 'X_val', 'y_test', 'y_train', 'y_val']

Understanding the data-

We have a file consisting of pixels of images of numbers. The data is to be split into 3 stages: train, validation and test sets. We have to implement the model which detects which numbers are present in those images.


The h5py package is a Pythonic interface to the HDF5 binary data format. HDF5 lets us store huge amounts of numerical data, and easily manipulate that data from NumPy.

# 2. Data fetching and understand the train/val/test splits

In [0]:
# Fetching the data

X_test = np.array(images_of_numbers_file['X_test'])
X_train = images_of_numbers_file['X_train']
X_val = images_of_numbers_file['X_val']
y_test = images_of_numbers_file['y_test']
y_train = images_of_numbers_file['y_train']
y_val = images_of_numbers_file['y_val']

In [5]:
# Understanding the shape of all splits

print("Shape of X_train \t", X_train.shape)
print("Shape of X_test \t", X_test.shape)
print("Shape of X_val  \t", X_val.shape)
print("Shape of y_train \t", y_train.shape)
print("Shape of y_test \t", y_test.shape)
print("Shape of y_val  \t", y_val.shape)

Shape of X_train 	 (42000, 32, 32)
Shape of X_test 	 (18000, 32, 32)
Shape of X_val  	 (60000, 32, 32)
Shape of y_train 	 (42000,)
Shape of y_test 	 (18000,)
Shape of y_val  	 (60000,)


In [6]:
# Understanding the datatype of all keys

print("Type of X_train\t", X_train.dtype)
print("Type of X_test \t", X_test.dtype)
print("Type of X_val  \t", X_val.dtype)
print("Type of y_train\t", y_train.dtype)
print("Type of y_test \t", y_test.dtype)
print("Type of y_val  \t", y_val.dtype)

Type of X_train	 float32
Type of X_test 	 float32
Type of X_val  	 float32
Type of y_train	 uint8
Type of y_test 	 uint8
Type of y_val  	 uint8


In [0]:
# Converting data into arrays and then, normalizing image data from 0-255 to 0-1
X_train = np.array(X_train)
X_test = np.array(X_test)
X_val = np.array(X_val)

X_train = X_train/255
X_test = X_test/255
X_val = X_val/255

# 3. Implement and apply an optimal K-Nearest Neighbor (KNN) classifier

In [0]:
# Flattening the data for KNN classifier

X_train_reshaped = np.reshape(X_train,(42000, 32*32))
X_test_reshaped = np.reshape(X_test,(18000, 32*32))

In [9]:
#Trying to find the right value of k:
list_opt = np.arange(1, 51)
accuracies = []

for Ks in list_opt:
    model_knn = KNeighborsClassifier(n_neighbors = Ks, algorithm = 'brute')
    model_knn = model_knn.fit(X_train_reshaped, y_train)
    
    # Evaluate the model and update the accuracies list
    y_pred = model_knn.predict(X_test_reshaped)
    acc_score = accuracy_score(y_test, y_pred)
    print(acc_score)
    accuracies.append(acc_score)

0.45916666666666667
0.43644444444444447
0.4617777777777778
0.48133333333333334
0.49016666666666664
0.5008888888888889
0.5070555555555556
0.5117777777777778
0.5124444444444445
0.5168333333333334
0.518
0.5203333333333333
0.5210555555555556
0.5219444444444444
0.5235555555555556
0.5256111111111111
0.5287222222222222
0.5266666666666666
0.5271666666666667
0.5277222222222222
0.5290555555555555
0.5294444444444445
0.5297777777777778
0.5295555555555556
0.5296666666666666
0.5323333333333333
0.5322777777777777
0.5319444444444444
0.5316111111111111
0.5327777777777778
0.5329444444444444
0.5333888888888889
0.5331111111111111
0.5333333333333333
0.5343333333333333
0.5356111111111111
0.5343888888888889
0.5363888888888889
0.5372222222222223
0.5381111111111111
0.5372777777777777
0.5393333333333333
0.5382222222222223
0.5385555555555556
0.5381666666666667
0.5398888888888889
0.5401666666666667
0.5394444444444444
0.5381111111111111
0.5383333333333333


In [10]:
# Finding the value of k that is getting the largest accuracy

i = int(np.argmax(accuracies))
print("k=", list_opt[i], "achieved highest accuracy of", accuracies[i] * 100, "on validation data")

k= 47 achieved highest accuracy of 54.016666666666666 on validation data


In [0]:
# Our final KNN model with k = 1

model_knn_2 = KNeighborsClassifier(n_neighbors = 47, algorithm = 'brute')
model_knn_2 = model_knn_2.fit(X_train_reshaped, y_train)
y_pred_2 = model_knn_2.predict(X_test_reshaped)

# 4. Print the classification metric report

In [12]:
# The classification report is as follows:

class_report = classification_report(y_test, y_pred_2)
print(class_report)

              precision    recall  f1-score   support

           0       0.45      0.71      0.55      1814
           1       0.45      0.74      0.56      1828
           2       0.65      0.53      0.59      1803
           3       0.50      0.44      0.47      1719
           4       0.65      0.65      0.65      1812
           5       0.55      0.41      0.47      1768
           6       0.53      0.43      0.48      1832
           7       0.69      0.62      0.65      1808
           8       0.49      0.39      0.43      1812
           9       0.55      0.47      0.51      1804

    accuracy                           0.54     18000
   macro avg       0.55      0.54      0.54     18000
weighted avg       0.55      0.54      0.54     18000



In [13]:
# Also, finding the accuracy and the confusion matrix to evaluate the model:

accuracy = accuracy_score(y_test, y_pred_2)
c_matrix = confusion_matrix(y_test, y_pred_2)

print("The accuracy is", round(accuracy*100, 2), "and the confusion matrix is\n", c_matrix)

The accuracy is 54.02 and the confusion matrix is
 [[1279   65   34   36   42   38   90   36   70  124]
 [ 106 1345   56   75   75   28   34   57   23   29]
 [  96  229  962   72   49   44   32  172   57   90]
 [ 101  277   87  761   58  154   41   64   94   82]
 [ 108  263   35   60 1180   16   51   19   31   49]
 [ 163  177   43  228   66  729  135   35  115   77]
 [ 316  126   30   50  143   95  794   23  195   60]
 [  99  204  108   69   42   34   46 1128   27   51]
 [ 254  128   63   91  104  106  226   24  699  117]
 [ 289  161   53   68   65   80   51   84  107  846]]


# 5. Implement and apply a deep neural network classifier including (feedforward neural network, RELU activations)

In [0]:
# Converting labels to one hot vectors

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [15]:
# Understanding the shape of all splits

print("Shape of X_train \t", X_train.shape)
print("Shape of X_test \t", X_test.shape)
print("Shape of X_val  \t", X_val.shape)
print("Shape of y_train \t", y_train.shape)
print("Shape of y_test \t", y_test.shape)
print("Shape of y_val  \t", y_val.shape)

Shape of X_train 	 (42000, 32, 32)
Shape of X_test 	 (18000, 32, 32)
Shape of X_val  	 (60000, 32, 32)
Shape of y_train 	 (42000, 10)
Shape of y_test 	 (18000, 10)
Shape of y_val  	 (60000, 10)


In [16]:
# Understanding the datatype of all keys

print("Type of X_train\t", X_train.dtype)
print("Type of X_test \t", X_test.dtype)
print("Type of X_val  \t", X_val.dtype)
print("Type of y_train\t", y_train.dtype)
print("Type of y_test \t", y_test.dtype)
print("Type of y_val  \t", y_val.dtype)

Type of X_train	 float32
Type of X_test 	 float32
Type of X_val  	 float32
Type of y_train	 float32
Type of y_test 	 float32
Type of y_val  	 float32


In [17]:
# Clearing out tensorflow memory
clear_session()

# Define deep neural network Model
model_dnn = Sequential()

#Reshape data from 2D to 1D -> 32x32 to 1024
model_dnn.add(Reshape((1024,),input_shape=(32,32,)))

# 1st hidden layer
model_dnn.add(Dense(200, activation='relu'))

# 2nd hidden layer
model_dnn.add(Dense(100, activation='relu'))

# Dropout layer
model_dnn.add(Dropout(0.25))

# 3rd hidden layer
model_dnn.add(Dense(60, activation='relu'))

# 4th hidden layer
model_dnn.add(Dense(30, activation='relu'))

# Output layer
model_dnn.add(Dense(10, activation='softmax'))

# Loss and Optimizer
model_dnn.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

# Training the model
model_dnn.fit(X_train, y_train, batch_size=32, nb_epoch=50, validation_data=(X_test, y_test))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 42000 samples, validate on 18000 samples
Epoch 1/50




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.





42000/42000 [==============================] - 6s 132us/step - loss: 2.2955 - acc: 0.1211 - val_loss: 2.2634 - val_acc: 0.1976
Epoch 2/50
42000/42000 [==============================] - 5s 114us/step - loss: 2.1524 - acc: 0.2071 - val_loss: 2.0307 - val_acc: 0.2642
Epoch 3/50
42000/42000 [==============================] - 5s 111us/step - loss: 1.8649 - acc: 0.3330 - val_loss: 1.6090 - val_acc: 0.4885
Epoch 4/50
42000/42000 [==============================] - 5s 107us/step - loss: 1.5944 - acc: 0.4527 - val_loss: 1.4809 - val_acc: 0.4921
Epoch 5/50
42000/42000 [==============================] - 5s 108us/step - loss: 1.4536 - acc: 0.5102 - val_loss: 1.4014 - val_acc: 0.5158
Epoch 6/50
42000/42000 [==============================] - 5s 110us/step - loss: 1.3641 - acc: 0.5468 - val_loss: 1.3089 - val_acc: 0.5582
Epoch 7/50
42000/42000 [==============================] - 4s 106us/step - loss: 1.2916 - acc: 0.5745 - val_loss: 1.1590 - val_acc: 0.6348
Epoch 8/50
42000/42000 [==================

In [18]:
model_dnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               205000    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 60)                6060      
_________________________________________________________________
dense_4 (Dense)              (None, 30)                1830      
_________________________________________________________________
dense_5 (Dense)              (None, 10)               

# 6. Understand and be able to implement (vectorized) backpropagation (cost stochastic gradient descent, cross entropy loss, cost functions)

In [19]:
# Clearing out tensorflow memory
clear_session()

# Define deep neural network Model
model_dnn_2 = Sequential()

#Reshape data from 2D to 1D -> 32x32 to 1024
model_dnn_2.add(Reshape((1024,),input_shape=(32,32,)))

# 1st hidden layer
model_dnn_2.add(Dense(200, activation='relu'))

# 2nd hidden layer
model_dnn_2.add(Dense(100, activation='relu'))

# Dropout layer
model_dnn_2.add(Dropout(0.25))

# 3rd hidden layer
model_dnn_2.add(Dense(60, activation='relu'))

# 4th hidden layer
model_dnn_2.add(Dense(30, activation='relu'))

# Output layer
model_dnn_2.add(Dense(10, activation='softmax'))

# Loss and Optimizer
sgd_optimizer = SGD(lr = 0.05)
model_dnn_2.compile(loss='categorical_crossentropy', optimizer=sgd_optimizer, metrics=['accuracy'])

# Training the model
model_dnn_2.fit(X_train, y_train, batch_size=32, nb_epoch=50, validation_data=(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 42000 samples, validate on 18000 samples
Epoch 1/50
42000/42000 [==============================] - 5s 115us/step - loss: 2.2969 - acc: 0.1119 - val_loss: 2.2342 - val_acc: 0.1674
Epoch 2/50
42000/42000 [==============================] - 5s 108us/step - loss: 2.0389 - acc: 0.2515 - val_loss: 1.7079 - val_acc: 0.3823
Epoch 3/50
42000/42000 [==============================] - 5s 109us/step - loss: 1.6417 - acc: 0.4211 - val_loss: 1.3514 - val_acc: 0.5343
Epoch 4/50
42000/42000 [==============================] - 5s 111us/step - loss: 1.4345 - acc: 0.5121 - val_loss: 1.5207 - val_acc: 0.5169
Epoch 5/50
42000/42000 [==============================] - 5s 109us/step - loss: 1.3046 - acc: 0.5706 - val_loss: 1.3912 - val_acc: 0.5258
Epoch 6/50
42000/42000 [==============================] - 5s 108us/step - loss: 1.2082 - acc: 0.6062 - val_loss: 1.0874 - val_acc: 0.6498
Epoch 7/50
42000/42000 [==============================] - 5s 110us/step - loss: 1.1419 - acc: 0.6323 - val_loss: 1.1031 - 

In [20]:
model_dnn_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               205000    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 60)                6060      
_________________________________________________________________
dense_4 (Dense)              (None, 30)                1830      
_________________________________________________________________
dense_5 (Dense)              (None, 10)               

# 7. Implement batch normalization for training the neural network

In [21]:
# Clearing out tensorflow memory
clear_session()

# Define deep neural network Model
model_dnn_3 = Sequential()

#Reshape data from 2D to 1D -> 32x32 to 1024
model_dnn_3.add(Reshape((1024,),input_shape=(32,32,)))

#Normalize the data
model_dnn_3.add(BatchNormalization())

# 1st hidden layer
model_dnn_3.add(Dense(200, activation='relu'))
model_dnn_3.add(BatchNormalization())

# 2nd hidden layer
model_dnn_3.add(Dense(100, activation='relu'))
model_dnn_3.add(BatchNormalization())

# Dropout layer
model_dnn_3.add(Dropout(0.25))

# 3rd hidden layer
model_dnn_3.add(Dense(60, activation='relu'))
model_dnn_3.add(BatchNormalization())

# 4th hidden layer
model_dnn_3.add(Dense(30, activation='relu'))
model_dnn_3.add(BatchNormalization())

# Output layer
model_dnn_3.add(Dense(10, activation='softmax'))

# Loss and Optimizer
model_dnn_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model
model_dnn_3.fit(X_train, y_train, batch_size=32, nb_epoch=50, validation_data=(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 42000 samples, validate on 18000 samples
Epoch 1/50
42000/42000 [==============================] - 15s 355us/step - loss: 1.7065 - acc: 0.4172 - val_loss: 1.1557 - val_acc: 0.6389
Epoch 2/50
42000/42000 [==============================] - 14s 331us/step - loss: 1.2307 - acc: 0.6068 - val_loss: 0.9818 - val_acc: 0.6871
Epoch 3/50
42000/42000 [==============================] - 14s 331us/step - loss: 1.1056 - acc: 0.6487 - val_loss: 0.8801 - val_acc: 0.7246
Epoch 4/50
42000/42000 [==============================] - 14s 332us/step - loss: 1.0216 - acc: 0.6751 - val_loss: 0.7935 - val_acc: 0.7580
Epoch 5/50
42000/42000 [==============================] - 14s 332us/step - loss: 0.9694 - acc: 0.6933 - val_loss: 0.7252 - val_acc: 0.7778
Epoch 6/50
42000/42000 [==============================] - 14s 334us/step - loss: 0.9091 - acc: 0.7122 - val_loss: 0.6952 - val_acc: 0.7861
Epoch 7/50
42000/42000 [==============================] - 14s 331us/step - loss: 0.8682 - acc: 0.7280 - val_loss: 0.

In [22]:
model_dnn_3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 1024)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_1 (Dense)              (None, 200)               205000    
_________________________________________________________________
batch_normalization_2 (Batch (None, 200)               800       
_________________________________________________________________
dense_2 (Dense)              (None, 100)               20100     
_________________________________________________________________
batch_normalization_3 (Batch (None, 100)               400       
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)              

In [0]:
# Prediction
y_pred_3 = model_dnn_3.predict(X_test)

In [24]:
# The classification report is as follows:

class_report = classification_report(y_test.argmax(axis=1), y_pred_3.argmax(axis=1))
print(class_report)

              precision    recall  f1-score   support

           0       0.88      0.91      0.89      1814
           1       0.83      0.89      0.86      1828
           2       0.90      0.88      0.89      1803
           3       0.83      0.82      0.83      1719
           4       0.87      0.90      0.89      1812
           5       0.85      0.86      0.86      1768
           6       0.90      0.83      0.86      1832
           7       0.88      0.90      0.89      1808
           8       0.85      0.83      0.84      1812
           9       0.87      0.84      0.86      1804

    accuracy                           0.87     18000
   macro avg       0.87      0.87      0.87     18000
weighted avg       0.87      0.87      0.87     18000



In [25]:
# Also, finding the accuracy and the confusion matrix to evaluate the model:

accuracy = accuracy_score(y_test.argmax(axis=1), y_pred_3.argmax(axis=1))
c_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred_3.argmax(axis=1))

print("The accuracy is", round(accuracy*100, 2), "% and the confusion matrix is\n", c_matrix)

The accuracy is 86.7 % and the confusion matrix is
 [[1643   34    8   17   25    1   19   18   19   30]
 [  21 1626   12   30   54    9    8   50   11    7]
 [  13   27 1591   36   24   15    3   42   21   31]
 [  16   32   37 1412   18   97   10   41   37   19]
 [  15   57   18   21 1630    5   14   11   10   31]
 [  15   16   13   71   14 1527   40    9   43   20]
 [  50   28   15   19   37   66 1526    9   64   18]
 [  15   65   24   20   15    8    9 1630   11   11]
 [  26   48   23   39   19   23   61   15 1505   53]
 [  53   30   29   39   31   35   12   18   41 1516]]


# 8. Understand the differences and trade-offs between traditional and NN classifiers with the help of classification metrics 

The accuracy of the KNN model is coming out to be 54.02%, whereas, the accuracy of the deep neural network model is coming out to be 86.7%.

The accuracy of KNN model is very poor as compared to the accuracy of the deep neural networks model.

The advantage of KNN model is that we do not need to do hyperparameter tuning. NN needs a lot of hyperparameter tuning compared to KNN.

KNN didn't work with large data. NN needs large training data to achieve an efficient accuracy.

In this case, we will get large computation cost for KNN model during runtime.

From the above classification metrics, we can see that KNN is definitely not a good choice for this image classification model and the use of DNN model is truly justified.

### Closing the dataset file

In [0]:
images_of_numbers_file.close()